# CIS 399 Homework 4: Human Stereotypes and Word Embeddings

## (replace this line with your name)

In this homework, you will work with non-contextual word vector representations. The main operations we will do are simply looking up a representation and computing (cosine) similarity between words. We will use these operations to predict personality stereotypes based on a single word description. 

All cells where code is required are marked with a "YOUR CODE HERE" comment. The point values for each code block are written in the header for the associated subsection.


## Part 1: The data (20 pts)

For this assignment you will be using data collected via Amazon's Mechanical Turk. Raters were asked what expectations they would have about the personality of someone described only by their profession or nationality; ratings were given on a 7-point scale for each of the Big Five personality traits listed below:
1. Openness to experience
2. Conscientiousness
3. Extroversion
4. Agreeableness
5. Neuroticism

In this homework, we will focus on the stereotype data collected for the first of the five personality traits, Openness. The 7 point scale described above ranges from -3 to 3, where -3 corresponds to falling on one end of the openness dimension (being conventional/uncreative) , 0 corresponds to neutral expectation and 3 corresponds to falling on the other end of the openness dimension (being open to new experiences/complex). Each descriptive word got about 30 ratings, from which the mean was computed as well as the statistical significance for that mean being non zero. 

During this assignment, we will use the data they collected to create a machine learning model that can predict the expected openness of a person from a single word description. The variable will predict is the average of human ratings. The features will be the similarities between the person descriptor (Russian, entertainer, neighbor) and descriptors of openness (intelligent, perceptive, analytical, reflective, curious, imaginative, creative, cultured, refined, sophisticated as one end of the spectrum and  unintelligent, imperceptive, unanalytical, unreflective, uninquisitie, unimaginative, uncreative, uncultured, unrefined, unsophisticated as the other end of the openness spectrum). If our system has predictions that are more accurate than random guesses, the word representations encode aspects of the human stereotype. 

### Section 1.1: Building word vector dict (10)

First, download the word vectors. We will be using GloVe word embeddings. Please go to the [GloVe Website](https://nlp.stanford.edu/projects/glove/) and download the file labeled "glove.6B.zip". This is be the file of word embeddings trained on the common crawl of a large number of webpages. These word representations are often used in language technology applications. Move the 300D file from the zip into your working directory and update the `vector_file_name` variable.

Complete the following function. It should read in the vector file and return a python dictionary. Each line of the file contains a word and a 300 dimension vector that represents that word. The word and each element of the vector are seperated by a single space. Your dictionary should map from word to numpy array of `floats`.

Please note this is a pretty large file (around 400,000 words), so this might take a few minutes to download and to read into python. If you would like some notion of progress when running this, we recommend that you keep track of how many words you have read and print out this number every 100,000 iterations.

In [ ]:
import numpy as np
import pandas as pd
import codecs

## YOUR CODE HERE
def build_word_vector_dictionary(vector_file):
    
    ## Use this to read the File. You can use `enumerate(f)` to iterate through the file line by line.
    with codecs.open(vector_file, 'r', 'utf-8') as f:
        pass


# Build the vector matrix using your function
vector_file_name = "PATH TO VECTOR FILE"
vectors = build_word_vector_dictionary(vector_file_name)

### Section 1.2: Read in human stereotype data (10)

Fill in this function. The collection of the stereotype data was described above.

In the file, each line contains three comma-separated entries: a word, the average human openness rating for that description, and the p-score shwoing the likelihood of this mean if there were no stereotype.

You function should return a dictionary that maps from each word to its bias with the following specifications:
1. You should normalize the word. This means you should strip it of any leading/trailing whitespace, convert it to lowercase, and replace all spaces with an underscore ('_') character.
2. You should only include words that have GloVe vector representations.
3. If the p-score is less than or equal to .05 (which would indicate statistical significance), you should set its score to be the average human rating for the word. Otherwise, set its score to be 0.

In [ ]:
## YOUR CODE HERE
def read_bias_data(bias_data, vectors):
    pass

bias_file_name = "./bias.csv"
bias = read_bias_data(bias_file_name, vectors)

## Part 2: Exploring the Benefits of Word Embeddings (5 pts)
Before working with the word embeddings, let's quickly explore one of the aspects of word embeddings that makes them so interesting and useful.

Below, we have included code to reduce the number of features of the vectors to just two features using [TNSE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html). This allows us to plot the vectors on a standard two axis graph, making them more understandable. Using this, we provide an example of plotting a few adjectives that follow this pattern: "slow" -> "slower". As you can see in the plot, the difference between all such pairs is fairly similar. This is because you can apply linear transformations to GloVe word embeddings. For example, `glove("dark") + (glove("slow") - glove("slower")` should yield a vector very close to `glove("darker")`.

Run our provided code to see the plot. Then, in a different cell, use our code to plot a different set of words that you think would be interesting to visualize. For example, you could try plotting a similar pattern for conjugating verbs, or perhaps two groups of words that you believe would cluster into distinct groups. You will analyze this plot in the short response.

In [ ]:
## RUN THIS CELL
## DO NOT MODIFY THIS CELL
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

words_to_plot = [ "dark", "darker", "slow", "slower", "strong", "stronger"]

def plot_words_in_2D(vectors, words_to_plot):
    # Get word vectors for each word
    words_v = [vectors[word] for word in words_to_plot]
    
    # Find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    ## Ideally, we'd train the t-SNE model on the whole dataset, but the current implementation is not very
    ## Scalable, so we will only train on the word vectors we want to reduce. This many shift some of the reductions.
    Y = tsne.fit_transform(words_v)

    # Split up into the two dimensions
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    
    # Plot each point with the word label
    plt.scatter(x_coords, y_coords)
    for label, x, y in zip(words_to_plot, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    
    # Set plot parameters and show the plot
    plt.xlim(x_coords.min()-10, x_coords.max()+10)
    plt.ylim(y_coords.min()-10, y_coords.max()+10)
    plt.show()
    
plot_words_in_2D(vectors, words_to_plot)

In [ ]:
## YOUR CODE HERE

## Part 3: Training a Model to Predict Stereotypes (40 pts)
Now that we have the data in the format we want, we will attempt to train a model to predict stereotypical expectations about personality. Rather than using the word vectors as features, however, we will create new features. Our features will be the (cosine) similarity between a given word and the list of personality descriptors. The personality descriptors that we will use in our exercise are from Goldberg's Big Five markers. We will then train a regression model based on these features to predict a word's score.

### Section 3.1: Creating the features (25)
To start with this task, we first have to write a function that will compute the features for a single word. This should be a function that takes in the vectors, a given word, and a list of words. Your function should compute the cosine similarity between the given word and every word in the list. You may use [SciKit-Learn's cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function to help with this computation. Your function should return a list of the cosine similarities. This means the length of the returned list should be the same as the length of the `words` parameter.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

## YOUR CODE HERE
def get_similarity(vectors, given_word, desc_words):
    pass


Below, we have provided you with `desc_words`. This is the list of personality descriptors based off which we will create our features. As stated above, these description words are based off the first of Goldberg's Big Five personality traits, Openness to experience.

In [ ]:
## RUN THIS CELL
## DO NOT MODIFY THIS CELL

desc_words = ['intelligent','perceptive','analytical','reflective','curious','imaginative',
              'creative','cultured','refined','sophisticated','unintelligent','unreflective',
              'unimaginative','uncreative','uncultured','unrefined','unsophisticated']


Now, we will create the entire feature set. To do this, you should write a function that takes in the vectors (`vectors`), the stereotype data (`bias`), and a list of words (`desc_words`) and returns an 2-d array called `features` and an array called `scores`.

Our goal is to create features for each word that we have stereotype data about, so we can train a regression model. This means, for each word that we have stereotype data, you should use your `get_similarity` function above to create the features from which we will predict the associated stereotype. These features should be a row in `features`, and the corresponding row in `scores` should hold the bias of that word.

In [ ]:
## YOUR CODE HERE
def create_features(vectors, bias, desc_words):
    pass
    
    
# WE need to define words for them (Need to filter to make sure we have all their word vectors first)
features, scores = create_features(vectors, bias, desc_words)

### Section 3.2: Predicting Stereotypes (10)
Now that we have a features and labels, let's train a linear regression model to predict the stereotype score. You should create a model using [SciKit-Learn's Linear Regression class](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) and train it using the features and scores.

In [ ]:
## YOUR CODE HERE

Now, use your regression model to make insample predictions on the features you created. 

We also want to analyze how accurate our predictions are. However, both our predictions and our true scores are real numbers which means we cannot use our error functions from previous homework. For this homework, we will use the Spearman correlation to see if we were able to accuractly predict the scores. You should use [SciPy's Spearman's correlation](https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.spearmanr.html) function to calculate this. Print out the correlation between your predictions and the true score.

In [ ]:
## YOUR CODE HERE

### Section 3.3: Plotting results (5)

Now, let's plot the true scores and the predicted scores to better visualize the accuracy of our classifier. Sort the two arrays by increasing value of true score. Thus true score will be strictly increasing, but predicted score may not be.

Then, plot both as line graphs with the score as the y-axis and the index of that score as the x-axis.

You hopefully should see that although there is a lot of variance in the predicted scores, there is a visual trend that supports we are predicting more positively for words with a positive label.

In [ ]:
## YOUR CODE HERE

## Part 4: Testing model (15 pts)
Now that we have trained a model and evaluated its training accuracy, let's find if the model generalizes well.

### Section 4.1: Loading in Test data (5)
We have provided another file with new person descriptors and the associated average of human ratings. Write a function, `read_test_data`, which does the following for each line in the file:
1. Normalize the word. This means you should strip it of any leading/trailing whitespace, convert it to lowercase, and replace all spaces with an underscore ('_') character.
2. Only include words that have GloVe vector representations.

Your output should be an array of cleaned words that have made it through your filtering process.

In [ ]:
## YOUR CODE HERE
def read_test_data(test_data, vectors):
    pass

test_file_name = "./bias_test.csv"
test_words = read_test_data(test_file_name, vectors)

### Section 4.2: Building features for Testing (5)
Now, let's create similar features for our test words. Write a function `create_test_features`, that outputs the features, as described in section 3.1, of the test_words.

In [ ]:
## YOUR CODE HERE
def create_test_features(vectors, test_words, desc_words):
    pass

test_features = create_test_features(vectors, test_words, desc_words)

### Section 4.3: Predicting Stereotypes for Unseen Words (5)
Now, use our model from Part 3 to predict the stereotype associated with each words. Print out each test word with its corresponding predicted score in ascending order of score. We will explore the meaning of these scores in the short response.

In [ ]:
## YOUR CODE HERE


## Part 5: Short answer Questions (20 pts)

**Q1**. Was your plot for Part 2 what you expected? If so, why did you predict the words to have that relationship? If not, why do you think the relationship was different than you expected?

Your answer to Q1 here.

**Q2**. Based on the predicted scores from Part 4.3, pick two words that you believe are predicted accurately and two words that you believe were not predicted accurately, i.e, two for which you'd expect people have the stereotype and two for which you'd not expect that.

Your answer to Q2 here.

**Q3**. Given the number of training examples, do you think using GloVe representations for the target group as features directly would be a reasonable approach. Justify your answer.

Your answer to Q3 here.

**Q4**. Do you think it is necessary to make changes in the word representations so such associations between personality and person descriptors do not exist. Justify your answer.

Your answer to Q4 here.